In [ ]:
from eppy import modeleditor
from eppy.modeleditor import IDF
from matplotlib import pyplot as plt
import os
import pandas as pd
import besos as besos
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, FieldSelector, FilterSelector, Parameter, expand_plist, wwr, CategoryParameter, GenericSelector
from besos.problem import EPProblem

#### Define the IDF, IDD and the weather file

In [ ]:
idd_file=' dir to EnergyPlus/Energy+.idd'
weather_file ='dir to epw file'

In [ ]:
building = ef.get_building('dir to idf')

In [ ]:
insulation = FieldSelector(class_name='Material', object_name='Cement/plaster/mortar - gypsum plaster_O.OO2', field_name='Thickness')
insulationPR = Parameter(selector=insulation, value_descriptor=CategoryParameter(options=[0.0001,0.05,0.2]),name='Insulation Thickness')
window_to_wall = wwr(RangeParameter(0.1,0.9))

parameters = [window_to_wall, insulationPR]

#### setting optimal found values as sample

In [ ]:
samples_temp =[]
samples_temp.append({'Window to Wall Ratio': 0.1,                     
                     'Insulation Thickness': 0.2,})
samples = pd.DataFrame.from_dict(samples_temp)
samples

In [ ]:
objectives = ['Electricity:Facility', 'DistrictHeating:Facility', 'DistrictCooling:Facility']

problem = EPProblem(parameters, objectives)
evaluator = EvaluatorEP(problem, building, out_dir=('Output directory'),
                        error_mode='Silent', err_dir='Error directory',
                        epw= weather_file)
outputs = evaluator.df_apply(samples, keep_input=True)    
outputs

In [ ]:
idf_file = 'idf directory'
IDF.setiddname(idd_file)
idf = IDF(idf_file,weather_file)

#### Performing simulation

In [ ]:
idf.run(readvars=True,output_directory=u'directory for output_EPPY')

#### Performing file management to extract desired results

In [ ]:
csv_columns = ['Date/Time',
              'BLOCK1:ZONE2:Zone Operative Temperature [C](Hourly)',
              'BLOCK1:ZONE1:Zone Operative Temperature [C](Hourly)',
              'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)',
              'DistrictHeating:Facility [J](Hourly)',
              'DistrictCooling:Facility [J](Hourly)',
              'Electricity:Facility [J](Hourly)']
              #'InteriorEquipment:Electricity [J](Hourly)']

In [ ]:
df_columns = {'Date/Time':'Date_Time',
              'BLOCK1:ZONE2:Zone Operative Temperature [C](Hourly)':'t_in_ZONE2',
              'BLOCK1:ZONE1:Zone Operative Temperature [C](Hourly)':'t_in_ZONE1',
              'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)':'t_out',
              'DistrictHeating:Facility [J](Hourly)':'power_heating',
              'DistrictCooling:Facility [J](Hourly)':'power_cooling' ,
              'Electricity:Facility [J](Hourly)':'power_electricity'}
              #'InteriorEquipment:Electricity [J](Hourly)':'power_InteriorEquipment'}

In [ ]:
idf_data =pd.read_csv('csv directory',usecols=csv_columns)

In [ ]:
idf_data = idf_data.rename(columns =df_columns)

In [ ]:
idf_data['t_in'] = idf_data[['t_in_ZONE2','t_in_ZONE1']].mean(axis=1)
idf_data.drop(['t_in_ZONE2', 't_in_ZONE1'], axis = 1)

In [ ]:
idf_data = idf_data.dropna()

In [ ]:
idf_data.reset_index(drop = True)

In [ ]:
idf_data['temp_diff'] =idf_data['t_in'] - idf_data['t_out']

In [ ]:
idf_data['Date_Time'] = '2020/' + idf_data['Date_Time'].str.strip()
idf_data['Date_Time'] = idf_data['Date_Time'].str.replace('24:00:00','23:59:59')

In [ ]:
idf_data['power_heating'] /= 3.6e6
idf_data['power_cooling'] /= 3.6e6
idf_data['power_electricity'] /= 3.6e6

In [ ]:
idf_data['power_heating'] = idf_data['power_heating'].apply(lambda x: round(x, 2))
idf_data['power_cooling'] = idf_data['power_cooling'].apply(lambda x: round(x, 2))
idf_data['power_electricity'] = idf_data['power_electricity'].apply(lambda x: round(x, 2))
idf_data['t_in'] = idf_data['t_in'].apply(lambda x: round(x, 2))
idf_data['t_out'] = idf_data['t_out'].apply(lambda x: round(x, 2))

In [ ]:
idf_data['total_power'] = idf_data['power_heating']+ idf_data['power_cooling'] #+idf_data['power_electricity']

In [ ]:
idf_data = idf_data[['Date_Time','t_in','t_out','temp_diff','power_heating','power_cooling','power_electricity','total_power']]

In [ ]:
idf_data.to_csv(path_or_buf='Final simulation Data csv directory',index=False)